In [1]:
import pandas as pd
import multiprocessing as mp
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
import time
import numpy as np
from multiprocessing import Pool
np.random.seed(seed=1991)
import pickle as pkl

In [2]:
def decader(x):
    return(x -x%10)
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]

NOUN (nouns),
VERB (verbs), ADJ (adjectives), ADV (adverbs),
PRON (pronouns), DET (determiners and articles),
ADP (prepositions and postpositions), NUM (numerals), CONJ (conjunctions), PRT (particles), ‘.’
(punctuation marks) and X

In [3]:
unigrams="a b c d e f g h i j k l m n o other p pos punctuation q r s t u v w x y z"
unigram_list=unigrams.split()
br_to_us=pd.read_excel("Book.xlsx",header=1)
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
np.random.shuffle(unigram_list)
spelling_replacement={'context':br_to_us_dict}
num_cores=mp.cpu_count() -1

In [4]:
def lemma_maker(x, y):
    #print(lemmatizer(x,y)[0])
    return lemmatizer(x,y)[0]

In [5]:
def reducer(dfs):
    df=dfs.copy()
    if df.empty==True:
        return df
    
    df['context'],df['pos']=df['context_pos'].str.split('_', 1).str
    df.replace(spelling_replacement,inplace=True)
    df['context']=np.vectorize(lemma_maker)(df['context'], df['pos'])
    df['context']=df['context']+"_"+df['pos']
    #df.drop(['pos'],axis=1,inplace=True)
    df=df.groupby(['context','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    return df

In [6]:
def chunked_dataset_extracter(dfs):
    #df=dfs.copy()
    dfs.columns=['context_pos','decade','count']
    df=dfs.query('decade > 1799').copy()
    df['decade']=np.vectorize(decader)(df['decade'])
    df=df.groupby(['context_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    df.context_pos=df.context_pos.str.lower()
    df=df.loc[df.context_pos.str.match("^['.a-z-]+_(noun|verb|adj|adv)$",na=False)]
    df=df.loc[~(df.context_pos.str.match("^.*['.-]{2,}.*$",na=False))]
    df=df.groupby(['context_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    #df=reducer(df)
    return df

In [8]:
def dataset_extracter(letter):
    CHUNKSIZE = 1_000_000
    print(f"Started with letter(s) {letter}")
    cur_time=time.time()
    total_df_shape=0
    df_list=[]
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-1gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quotechar='"',usecols=[0,1,2],chunksize=CHUNKSIZE)
    for df in dfs:
        total_df_shape+=df.shape[0]
        df_list.append(chunked_dataset_extracter(df))
    #print(total_df_shape)
    complete_df=pd.concat(df_list,ignore_index=True)
    #print(complete_df.shape[0])
    total_contexts=reducer(complete_df)
    
    after_shape=total_contexts.shape[0]
    print(f"Finished with letter(s) {letter} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f"Letter(s) {letter} took time {(time.time()-cur_time):0.2f} seconds")
    print("\n")
    return total_contexts

In [9]:
dfs=dataset_extracter('j')

Started with letter(s) j
Finished with letter(s) j ; Before : 17008269, After : 884073 Change in percentage : 94.80%
Letter(s) j took time 86.91 seconds




In [17]:
dfs.loc[dfs.context.str.contains('_')]

,context,decade,count
0,j'_adj,1800,7
1,j'_adj,1810,10
2,j'_adj,1820,8
3,j'_adj,1830,8
4,j'_adj,1840,12
5,j'_adj,1850,18
6,j'_adj,1860,12
7,j'_adj,1870,22
8,j'_adj,1880,18
9,j'_adj,1890,38


In [18]:
context_list=[]
pool = Pool(num_cores)
#print("Started with letter "+str(unigram_list))
for temp_contexts in pool.imap_unordered(dataset_extracter,unigram_list):
    context_list.append(temp_contexts)
        
pool.close()
pool.join()
contexts = pd.concat(context_list,ignore_index=True,sort=False)
contexts.decade=contexts.decade.astype("int32")
contexts_df=contexts.pivot_table(values='count',columns='decade',index='context',aggfunc=np.sum)

Started with letter(s) e
Started with letter(s) j
Started with letter(s) y
Started with letter(s) i
Started with letter(s) m
Started with letter(s) x
Started with letter(s) w
Started with letter(s) g
Started with letter(s) f
Started with letter(s) q
Started with letter(s) d
Started with letter(s) pos
Started with letter(s) r
Started with letter(s) b
Started with letter(s) punctuation
Started with letter(s) l
Started with letter(s) h
Started with letter(s) c
Started with letter(s) k
Started with letter(s) p
Started with letter(s) n
Started with letter(s) v
Started with letter(s) o
Started with letter(s) s
Started with letter(s) a
Started with letter(s) z
Started with letter(s) t
Started with letter(s) u
Started with letter(s) other
Finished with letter(s) pos ; Before : 4250, After : 0 Change in percentage : 100.00%
Letter(s) pos took time 0.24 seconds


Finished with letter(s) other ; Before : 97674, After : 0 Change in percentage : 100.00%
Letter(s) other took time 0.36 seconds


Fini

In [20]:
contexts_df

decade,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890,...,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000
context,,,,,,,,,,,,,,,,,,,,,
'_adv,103972.0,125832.0,186022.0,240938.0,223659.0,345615.0,399334.0,633392.0,816868.0,901098.0,...,820092.0,854196.0,884490.0,750755.0,1402187.0,2833805.0,3900740.0,5487186.0,8571111.0,14614001.0
'_noun,19951.0,30788.0,43962.0,73431.0,139412.0,255480.0,206652.0,233312.0,255974.0,258352.0,...,201149.0,179398.0,167217.0,211614.0,343942.0,699959.0,842184.0,937638.0,1231765.0,1692352.0
'_verb,233902.0,287827.0,385176.0,478464.0,571687.0,909050.0,1085939.0,1435716.0,1969267.0,2915489.0,...,4965545.0,4709221.0,4917721.0,4969736.0,7490454.0,12289013.0,20162725.0,36118111.0,68141327.0,127572458.0
'c_noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,...,NaN,2.0,2.0,1.0,2.0,6.0,9.0,11.0,26.0,67.0
'd_noun,1.0,NaN,1.0,4.0,3.0,2.0,5.0,8.0,7.0,6.0,...,23.0,34.0,22.0,34.0,53.0,103.0,267.0,400.0,1084.0,2112.0
'h_noun,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,2.0,3.0,2.0,1.0,11.0,10.0,5.0,10.0
'i_noun,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,...,2.0,1.0,NaN,3.0,1.0,2.0,6.0,6.0,10.0,15.0
'j_noun,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,...,1.0,1.0,3.0,2.0,2.0,5.0,9.0,15.0,11.0,24.0
'll_adj,3.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2.0,1.0,...,3.0,9.0,NaN,9.0,12.0,21.0,29.0,73.0,114.0,188.0


In [22]:
total_freq_all=contexts_df.sum(axis=1)
total_freq_no_2000=contexts_df.drop(2000,axis=1).sum(axis=1)
all_decade_freq=contexts_df.dropna().sum(axis=1)

all_decade_contents=contexts_df[contexts_df>10]
all_decade_free_min_10=all_decade_contents.dropna().sum(axis=1)

In [23]:
top_total_freq_all=total_freq_all.sort_values(ascending=False).head(50_000).index.tolist()
top_total_freq_no_2000=total_freq_no_2000.sort_values(ascending=False).head(50_000).index.tolist()
top_all_decade_freq=all_decade_freq.sort_values(ascending=False).head(50_000).index.tolist()
top_all_decade_free_min_10=all_decade_free_min_10.sort_values(ascending=False).head(50_000).index.tolist()

In [24]:
len(set(top_total_freq_all).intersection(top_total_freq_no_2000))

48218

In [25]:
len(set(top_all_decade_freq).intersection(top_all_decade_free_min_10))

44488

In [29]:
len(set(top_total_freq_all).intersection(top_all_decade_free_min_10))

39664

In [21]:
chosen_context=contexts.sort_values('count',ascending=False).head(50_000)
chosen_context.context["of"]

KeyError: 'of'

In [32]:
top_all_decade_free_min_10[:100]

['be_verb',
 'have_verb',
 'not_adv',
 'do_verb',
 'can_verb',
 'may_verb',
 'when_adv',
 'make_verb',
 'would_verb',
 'will_verb',
 'oth_adj',
 'time_noun',
 'see_verb',
 'say_verb',
 'so_adv',
 'man_noun',
 'take_verb',
 'also_adv',
 'such_adj',
 'use_verb',
 'give_verb',
 'only_adv',
 'more_adv',
 'go_verb',
 'could_verb',
 'year_noun',
 'state_noun',
 'then_adv',
 'know_verb',
 'should_verb',
 'come_verb',
 'find_verb',
 'first_adj',
 'great_adj',
 'many_adj',
 'as_adv',
 'good_adj',
 'people_noun',
 'now_adv',
 'where_adv',
 'way_noun',
 'even_adv',
 'very_adv',
 'must_verb',
 'life_noun',
 'day_noun',
 'new_adj',
 'more_adj',
 'how_adv',
 'same_adj',
 'part_noun',
 'case_noun',
 "'_verb",
 'work_noun',
 'get_verb',
 'become_verb',
 'think_verb',
 'system_noun',
 'most_adv',
 'high_adj',
 'well_adv',
 'child_noun',
 'show_verb',
 'however_adv',
 'new_noun',
 'own_adj',
 'number_noun',
 'world_noun',
 'woman_noun',
 'large_adj',
 'call_verb',
 'long_adj',
 'just_adv',
 'power_noun'

Four extra columns are added, which are:

$cf$ : Collection frequency, which is the log of the sum of the term across decades, i.e. log(sum(term).

$presence$ : Number of decades a term is present in.

$pattern$ : A binary representation of a word. 1 if the $word$ exists in a decade, 0 otherwise.

In [34]:
pkl.dump(top_all_decade_free_min_10,open( "contexts.pkl", "wb" ) )